In [1]:
import gym
import tensorflow as tf
import numpy as np
from tqdm import tqdm
from collections import deque

In [2]:
# keras model approach
from tensorflow.keras import Model,Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dense, Input
from tensorflow.keras.activations import relu

In [3]:
# ADDING THE CODE SO THAT TENSORFLOW DOES NOT EAT THE WHOLE GPU MEMORY
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.keras.backend.set_floatx('float32')

In [4]:
env = gym.make('MountainCar-v0')
env.action_space

Discrete(3)

In [5]:
adam = tf.keras.optimizers.Adam(learning_rate = 0.001)

In [6]:
def model_keras():
    
    inputs = Input(shape=(2,))
    
    x = Dense(100,activation='relu',kernel_initializer="glorot_uniform")(inputs)
    x = BatchNormalization()(x)
    x = Dense(100,activation='relu',kernel_initializer="glorot_uniform")(x)
    x = BatchNormalization()(x)
    x = Dense(100,activation='relu',kernel_initializer="glorot_uniform")(x)
    x = BatchNormalization()(x)
    x = Dense(100,activation='relu',kernel_initializer="glorot_uniform")(x)
    x = BatchNormalization()(x)
    output = Dense(3,activation='linear',kernel_initializer="glorot_uniform")(x)
    model = Model(inputs=inputs, outputs=output, name="RL_Value_Function")
    
    print(model.summary())
    
    model.compile(optimizer=adam,loss='mean_squared_error',metrics=['mean_squared_error'])
    
    return model
# model 2 is the target model
model_1 = model_keras()
model_2 = model_keras()

Model: "RL_Value_Function"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense (Dense)                (None, 100)               300       
_________________________________________________________________
batch_normalization (BatchNo (None, 100)               400       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
batch_normalization_1 (Batch (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)         

In [7]:
def custom_loss(y_true,y_pred):
    return tf.keras.losses.mean_squared_error(y_true,y_pred)

In [8]:
# lets build some memory into the model to perform decorrelated batch updates
# this is TD learning

# so apparantly the NN has to be adjusted only according to the action taken by it
# for example if action 2 is taken then only the weights for action 2 should be changed
# for this reason we should only update the q_2 vector with regarding the chosen action
# rewards will also be added to that action only
# q_2 is what we thought the value of the state will be after doing action
# we also add the reward and make this the target for the NN

replay_batch = deque(maxlen = 3000)

warmup = 10 #will start training after these many episodes have passed

# to balance exploration
epsilon = {
"epsilon" : 1.0,
"epsilon_decay": 0.999,
"epsilon_min":0.01,
}

In [9]:
import random
random.seed(2020)

@tf.function
def train_step(model,new_state,new_rew,old_q,lamb): 
    
    with tf.GradientTape() as tape:
        # logits is the forward pass
        logits = model(new_state, training=True)
        
        loss_value = custom_loss(create_y_true(logits,new_rew,lamb),old_q)
    
    #we retrieve the gradients
    grads = tape.gradient(loss_value, model.trainable_weights)
    
    #THIS IS ONE STEP OF GRAD DESCENT (Minimizes the loss)
    adam.apply_gradients(zip(grads, model.trainable_weights))
    #model.fit(x = inputs,y = outputs,batch_size = 1,epochs = 1,verbose = 0)

#@tf.function
def batch_train(model_1,model_2,gamma,batch_size,epsilon):
    
    #decaying the exploration
    if epsilon['epsilon'] > epsilon['epsilon_min']:
         epsilon['epsilon'] =  epsilon['epsilon'] * epsilon['epsilon_decay']
   
    batch = random.sample(replay_batch,batch_size)
    
    batch_reward = []
    batch_action = []
    batch_done = []
    
    batch_current_state = np.zeros((batch_size, 2))
    batch_next_state = np.zeros((batch_size, 2))

    for i in range(batch_size):
        batch_reward.append(batch[i][2])
        batch_action.append(batch[i][1])
        batch_current_state[i] = batch[i][0]
        batch_next_state[i] = batch[i][3]
        batch_done.append(batch[i][4])
    #lets calculate the next state value as the current value will be calculated in 
    # in gradient tape
    
    next_q = model_2.predict(batch_next_state)
    
    max_q = []
    for i in next_q:
        max_q.append(max(i))
    max_q = np.array(max_q,dtype = 'float32')
    
    target = batch_reward + gamma*max_q # this is the Q learning Target
    
    logits = model_1(batch_current_state)
    
    q_target = np.array(logits)

    # replacing the q values of logits for which the action is taken
    # as only those have to be updated
    
    # VERY IMPORTANT NOTE, IF THE EPISODE ENDS THE DONE VALUE BECOMES TRUE
    # IT IS VERY IMPORTANT THAT THE NN UPDATES TOWARDS THIS TRUE VALUE RATHER THAN
    # ITS OWN THINKING VALUE (r + gamma*max(action)) THAT WE USE FOR ALL
    # NON TERMINAL REWARDS 
    # THIS MAKES OR BREAKS THE NETWORK VERY VERY IMPORTANT
    for i in range(batch_size):
        q_target[i][batch_action[i]] = target[i]
        if batch_done[i]:
            q_target[i][batch_action[i]] = batch_reward[i]
    
    model_1.fit(batch_current_state, q_target, batch_size=batch_size,
                       epochs=1, verbose=0)
    
    '''with tf.GradientTape() as tape:
        # logits is the forward pass
        logits = model(batch_current_state, training=True)
        
        q_target = np.array(logits)
        
        # replacing the q values of logits for which the action is taken
        # as only those have to be updated
        q_target[:,batch_action] = target
        # calculating the loss
        loss_value = custom_loss(q_target,logits)
    
    #we retrieve the gradients
    grads = tape.gradient(loss_value, model.trainable_weights)
    
    #THIS IS ONE STEP OF GRAD DESCENT (Minimizes the loss)
    adam.apply_gradients(zip(grads, model.trainable_weights))'''

def policy(q_vals,eps):
    # lets implement a policy which decays
    if np.random.rand() <= eps:  
        return random.randrange(3)
    else:
        action = np.argmax(q_vals[0])
        return action
def update_target_network():
    model_2.set_weights(model_1.get_weights())

In [ ]:
global_steps = 0
# to have same networks in the starting
update_target_network()

for i in tqdm(range(500)):
    observation = env.reset()
    observation = np.expand_dims(observation, axis=0)
    done = False
    total_reward = 0
    for j in range(1000):        
        #storing the current state
        state_1 = observation
        
        # this is the current q values
        q_state = model_1(observation)
        
        action = policy(q_state,epsilon["epsilon"]) # current action
        observation,reward,done,info = env.step(action)
        
        # calculating the total reward
        total_reward = total_reward + reward

        observation = np.expand_dims(observation, axis=0)
        state_2 = observation
        state_reward = reward

        if j<990 and done:
            state_reward = 1000
        if j>998 and done == False:
            reward = -100
        
        replay_batch.append((state_1,action,state_reward,state_2,done))
        
        if i>warmup:
            batch_train(model_1,model_2,0.99,64,epsilon)
            global_steps = global_steps + 1
            
        if done:
            update_target_network()
            break
            
    print(total_reward)

  0%|          | 0/500 [00:00<?, ?it/s]


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



  0%|          | 1/500 [00:01<10:24,  1.25s/it]

-200.0


  0%|          | 2/500 [00:02<09:54,  1.19s/it]

...

-200.0


 71%|███████   | 355/500 [2:26:35<29:45, 12.31s/it]

-200.0


 71%|███████   | 356/500 [2:26:47<29:35, 12.33s/it]

-200.0


In [ ]:
env.close()

In [ ]:
#lets test the nn
for i in tqdm(range(500)):
    observation = env.reset()
    observation = np.expand_dims(observation, axis=0)
    total_reward = 0
    for j in range(1000):
        env.render()
        nn_out = model_2.predict(observation)
        action = policy(nn_out,0)
        print(nn_out[0])
        print(action)
        observation,reward,done,info = env.step(action)
        observation = np.expand_dims(observation, axis=0)
        total_reward = total_reward + reward
        if done:
            print("episode ended")
            break
env.close()

In [ ]:
env.close()

In [ ]:
# random action
for i in tqdm(range(50)):
    observation = env.reset()
    observation = np.expand_dims(observation, axis=0)
    for j in range(1000):
        env.render()
        observation,reward,done,info = env.step(env.action_space.sample())
        print(reward,done)
        if done:
            break